# Direct Model
## Setup

In [1]:
from load_data import load_bolete_data, BoleteDataset

import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

import torch
from torch import nn, optim
import torch.nn.functional as F
import torchvision.transforms as T
import torchvision.models as models

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

data = load_bolete_data()
print()
for k in data.keys():
    print(k, np.shape(data[k]))

ItemsViewHDF5(<HDF5 file "bolete.h5" (mode r)>)

bolete-characteristics (38, 1873)
bolete-edibility (5, 1873)
bolete-images (3, 512, 512, 1873)
bolete-labels (1873,)


## Partition Data
Divide data in train and held-out test partitions

In [2]:
sss_tt = StratifiedShuffleSplit(n_splits=1, test_size=0.3, train_size=0.7, random_state=0)
X, y = data["bolete-images"].T, data["bolete-labels"].T
for train_idx, test_idx in sss_tt.split(X, y):
    train_x = X[train_idx]
    train_y_sp = data["bolete-labels"].T[train_idx].astype(np.int64)
    train_y_ed = data["bolete-edibility"].T[train_idx]
    train_y_ch = data["bolete-characteristics"].T[train_idx]

    test_x = X[test_idx]
    test_y_sp = data["bolete-labels"].T[test_idx].astype(np.int64)
    test_y_ed = data["bolete-edibility"].T[test_idx]
    test_y_ch = data["bolete-characteristics"].T[test_idx]


In [3]:
M = np.size(np.unique(data["bolete-labels"]))
N, H, W, C = np.shape(data["bolete-images"].T)
print("Number of species:", M)
print("N = {}, H = {}, W = {}, C = {}".format(N, H, W, C))

Number of species: 178
N = 1873, H = 512, W = 512, C = 3


## Create DataLoader

In [ ]:
train = BoleteDataset(train_x, train=True, download=True,
                             transform=transform)

## Setup PyTorch

In [8]:


USE_GPU = False
dtype = torch.float32 # we will be using float throughout this tutorial
# dtype = torch.cuda.FloatTensor

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cpu


## Define models

In [5]:
def simple_model():
    # copy final model from ass igment 2
    class Flatten(nn.Module):
        def forward(self, x):
            return flatten(x)

    channel_1 = 32
    channel_2 = 24
    channel_3 = 16
    hidden_dim = 150
    torch.manual_seed(0)
    learning_rate = 3e-3 # 1e-2

    model = nn.Sequential(
        nn.Conv2d(C, channel_1, kernel_size=5, padding=2),
        # nn.GroupNorm(4,channel_1),
        nn.BatchNorm2d(channel_1),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Conv2d(channel_1, channel_2, kernel_size=3, padding=1),
        # nn.GroupNorm(4,channel_2),
        nn.BatchNorm2d(channel_2),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Conv2d(channel_2, channel_3, kernel_size=3, padding=1),
        # nn.GroupNorm(4,channel_3),
        nn.BatchNorm2d(channel_3),
        nn.ReLU(),
        nn.Dropout(0.1),
        Flatten(),
        nn.Linear(channel_3 * H * W, hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim, M)
    )
    optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                        momentum=0.8, nesterov=True)
    return model, optimizer


## Cross Validate on training data

In [11]:
from train import * 

folds = 1
epochs = 1
sss = StratifiedShuffleSplit(n_splits=folds, random_state=0, test_size=0.2, train_size=0.8)

for train_index, val_index in sss.split(train_x, train_y_sp):
    k_train_x = torch.from_numpy(train_x[train_index])
    k_train_y = torch.from_numpy(train_y_sp[train_index])

    k_val_x = torch.from_numpy(train_x[val_index])
    k_val_y = torch.from_numpy(train_y_sp[val_index])
    
    print(np.shape(k_train_x), np.shape(k_train_y),
          np.shape(k_val_x), np.shape(k_val_y), sep="\n")

    model, optimizer = simple_model()

    train_model(
        model, 
        optimizer, 
        k_train_x, 
        k_train_y, 
        k_val_x, 
        k_val_y, 
        loss_fcn=F.cross_entropy,
        batch_size=32,
        epochs=epochs, 
        device=device,
        dtype=dtype
    )


torch.Size([1048, 512, 512, 3])
torch.Size([1048])
torch.Size([263, 512, 512, 3])
torch.Size([263])
